In [1]:
# Analysis of cytokine perturbation scores

In [3]:
import scanpy as sc
import pandas as pd
import numpy as np

import squidpy as sq
import matplotlib.pyplot as plt
from matplotlib import rcParams

ModuleNotFoundError: No module named 'squidpy'

In [ ]:
sc.settings.figdir = "./figures_cytokines/"

!mkdir figures_cytokines

In [ ]:
adata = sc.read_h5ad("h5ad/scvi/xenium5k_scVI_CellCharter.h5ad")

In [ ]:
adata

In [ ]:
np.sum(adata.obs.annot == 'nan')

In [ ]:
adata.obs.annot[adata.obs.annot == 'nan'] = pd.NA

In [ ]:
np.sum(adata.obs.annot == 'nan')

In [ ]:
adata.obs.annot = adata.obs.annot.cat.remove_categories('nan')

# Load bulk RNA-seq signatures

In [ ]:
# Parses DESeq differential expression table returning rows with significant genes
def load_de_genes(fn, sep=',', max_p=0.01, min_lfc=2, max_n=1000):
    de_df = pd.read_csv(fn, sep=sep, index_col=0)
    de_df = de_df.where(lambda x: (x.padj < max_p) & (x.log2FoldChange > min_lfc)).dropna().sort_values(['padj', 'log2FoldChange'], ascending=[True, False])
    return de_df.iloc[0:max_n, :]

def load_de_genes_down(fn, sep=',', max_p=0.01, max_lfc=-2, max_n=1000):
    de_df = pd.read_csv(fn, sep=sep, index_col=0)
    de_df = de_df.where(lambda x: (x.padj < max_p) & (x.log2FoldChange < max_lfc)).dropna().sort_values(['padj', 'log2FoldChange'], ascending=[True, False])
    return de_df.iloc[0:max_n, :]

In [ ]:
#!ls ../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_v2/csv/*.csv

In [ ]:
# # Load DESeq2 differential expression statistics
conditions = [
#    '17uT_over_unstim',
#    'Belinostat_over_DMSO',
    'Belinostat_over_unstim',
    'Combo_over_unstim',
    'GSKJ4_over_unstim',
    'Belinostat_over_DMSO',
    'GSKJ4_over_DMSO',
    '[inverse]_unstim_over_DMSO'

]

In [ ]:
# # Read tables, store in dictionary
# de_tabs_all = {
#     key: pd.read_csv("../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_batch2_v2/DESeq2_" + key + ".csv", index_col=0) for key in conditions
# }

In [ ]:
#de_tabs_all

In [ ]:
# Load gene tables of differentially up-regulated genes
de_tabs_sig = {
    key: load_de_genes("../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_batch2_v2/DESeq2_" + key + ".csv", min_lfc=0.0) for key in conditions
}

In [ ]:
de_tabs_sig_down = {
    key: load_de_genes_down("../fibroblast_reanalysis_bulk_DE_signature/bulkRNA_DE_batch2_v2/DESeq2_" + key + ".csv", max_lfc=0.0) for key in conditions
}

In [ ]:
# Count number of significant genes
[de_tabs_sig[key].shape[0] for key in de_tabs_sig]

In [ ]:
de_tabs_sig_down['Belinostat_over_unstim'].head(10)

In [ ]:
#de_tabs_sig

In [ ]:
# Score signatures
for rstim, rup in de_tabs_sig.items():
    g = rup.index.to_list()
    
    # Subset to overlap only
    g = set(g).intersection(adata.var.index)
    
    print(rstim, ": ", len(g))
    sc.tl.score_genes(adata, g, ctrl_size=len(g), score_name=f'{rstim}_upreg')

# Scale
for rstim in de_tabs_sig.keys():
    x = adata.obs[f'{rstim}_upreg'].values
    adata.obs[f'{rstim}_upreg_scaled'] = (x-x.mean()) / np.std(x)

In [ ]:
# Score signatures
for rstim, rup in de_tabs_sig_down.items():
    g = rup.index.to_list()
    
    # Subset to overlap only
    g = set(g).intersection(adata.var.index)
    
    print(rstim, ": ", len(g))
    sc.tl.score_genes(adata, g, ctrl_size=len(g), score_name=f'{rstim}_downreg')

# Scale
for rstim in de_tabs_sig.keys():
    x = adata.obs[f'{rstim}_downreg'].values
    adata.obs[f'{rstim}_downreg_scaled'] = (x-x.mean()) / np.std(x)

In [ ]:
# for rstim in rna_up_genes.keys():
#     x = adata.obs[f'{rstim}_upreg'].values
#     adata.obs[f'{rstim}_upreg_scaled'] = (x-x.mean()) / np.std(x)

In [ ]:
adata

In [ ]:
# Plot standardized scores
#rna_up_genes_show = rna_up_genes.copy()
#del rna_up_genes_show['IL6']
#del rna_up_genes_show['IL22']

sc.set_figure_params(dpi_save=300)

#sk.set_figsize((3, 3))
sc.pl.embedding(
    adata,
    basis='X_umap',
    color=[f'{rstim}_upreg_scaled' for rstim in de_tabs_sig.keys()] + ['annot'],
    ncols=5,
    #cmap='viridis',
    palette=sc.pl.palettes.default_102,
    cmap='RdBu_r',
    hspace=1.0,
    title=[f'{rstim}_upreg, n={de_tabs_sig[rstim].shape[0]}' for rstim in de_tabs_sig.keys()] + ['Cell type'],
    vmin=-4, vmax=4,
    #vcenter=0,
#    save="_bulkDE_signature_projection_scaled_v3.pdf"
)

In [ ]:
sc.pl.matrixplot(
    adata, [f'{rstim}_upreg_scaled' for rstim in de_tabs_sig.keys()], groupby='annot', figsize=(13, 2.0), swap_axes=True,
    colorbar_title="Mean cytokine score",
    cmap='RdBu_r', vmin=-2, vmax=2,
 #   save="cytokines_cell_type.pdf"
)

In [ ]:
de_tabs_sig.keys()

In [ ]:
sc.pl.matrixplot(
    adata,
    [
        'GSKJ4_over_DMSO_upreg_scaled',
        'Belinostat_over_DMSO_upreg_scaled',
        'GSKJ4_over_DMSO_downreg_scaled',
        'Belinostat_over_DMSO_downreg_scaled',
        '[inverse]_unstim_over_DMSO_upreg_scaled',
        '[inverse]_unstim_over_DMSO_downreg_scaled'
    ], groupby='annot', figsize=(13, 2.0), swap_axes=True,
    colorbar_title="Mean pert. score",
    cmap='RdBu_r', vmin=-2, vmax=2,
    save="treatment_pertubation_cell_type.pdf"
)

In [ ]:
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
plt.rcParams["figure.figsize"] = (12, 5)

sc.pl.violin(
    adata,
    [
        'GSKJ4_over_DMSO_upreg_scaled',
        'Belinostat_over_DMSO_upreg_scaled',
        'GSKJ4_over_DMSO_downreg_scaled',
        'Belinostat_over_DMSO_downreg_scaled',
        '[inverse]_unstim_over_DMSO_upreg_scaled',
        '[inverse]_unstim_over_DMSO_downreg_scaled'
    ],
    groupby='annot',
    stripplot=False,
    figsize=(60, 6.0),
    #swap_axes=True,
    colorbar_title="Mean pert. score",
    rotation=90,
#    cmap='RdBu_r', vmin=-2, vmax=2,
    save="_treatment_pertubation_cell_type.pdf"
)

In [ ]:
plt.rcParams["figure.figsize"] = (5, 5)

select_cells = ["Myofibroblast-SMC", "Myofibroblast_1", "Myofibroblast_2", "Myofibroblast_3", "Pericyte_1", "Pericyte_2", "S1_1", "S1_2", "S2", "S3_EGR3", "S3_crypt", "S3_intermediate", "S3_lamina propria", "S3_oral_mucosa-like", "S3x_1", "S3x_2", "S4", "S5", "SMC_ANO1", "SMC_CAPN3", "SMC_PPLP2", "vSMC"]

adata_sub = adata[adata.obs.annot.isin(select_cells)]
sc.pl.violin(
    adata_sub,
    [
        'GSKJ4_over_DMSO_upreg_scaled',
        'Belinostat_over_DMSO_upreg_scaled',
        'GSKJ4_over_DMSO_downreg_scaled',
        'Belinostat_over_DMSO_downreg_scaled',
        '[inverse]_unstim_over_DMSO_upreg_scaled',
        '[inverse]_unstim_over_DMSO_downreg_scaled'
    ],
    groupby='annot',
    stripplot=False,
    #swap_axes=True,
    colorbar_title="Mean pert. score",
    rotation=90,
#    cmap='RdBu_r', vmin=-2, vmax=2,
    save="_treatment_pertubation_cell_type_select.pdf"
)

In [ ]:
sc.pl.matrixplot(
    adata, [f'{rstim}_upreg_scaled' for rstim in de_tabs_sig.keys()], groupby='spatial_cluster18', figsize=(5, 2.0), swap_axes=True,
    colorbar_title="Mean cytokine score",
    cmap='PRGn', vmin=-2, vmax=2,
#    save="cytokines_CellCharter_niches.pdf"
)

In [ ]:
sc.pl.matrixplot(
    adata, [
        'GSKJ4_over_DMSO_upreg_scaled',
        'Belinostat_over_DMSO_upreg_scaled',
        'GSKJ4_over_DMSO_downreg_scaled',
        'Belinostat_over_DMSO_downreg_scaled',
        '[inverse]_unstim_over_DMSO_upreg_scaled',
        '[inverse]_unstim_over_DMSO_downreg_scaled'
    ], groupby='spatial_cluster18', figsize=(5, 2.0), swap_axes=True,
    colorbar_title="Mean cytokine score",
    cmap='PRGn', vmin=-2, vmax=2,
#    save="cytokines_CellCharter_niches.pdf"
)

In [ ]:
# rcParams["figure.figsize"] = (8, 4)
# sc.pl.violin(adata, [f'{rstim}_upreg_scaled' for rstim in rna_up_genes_show.keys()], groupby='spatial_cluster18', )

In [ ]:
adata

In [ ]:
# Formating
adata.obs.sample = adata.obs['sample']

In [ ]:
for column in [f'{rstim}_upreg_scaled' for rstim in de_tabs_sig.keys()]:
    print(column)

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=300)

sq.pl.spatial_scatter(
    adata, 
    color='Belinostat_over_DMSO_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
    save="spatial/Belinostat_over_DMSO_upreg_scaled.pdf", dpi=300
)

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=300)

sq.pl.spatial_scatter(
    adata, 
    color='GSKJ4_over_DMSO_upreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
    save="spatial/GSKJ4_over_DMSO_upreg_scaled.pdf", dpi=300
)

In [ ]:
#adata

In [ ]:
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=300)

sq.pl.spatial_scatter(
    adata, 
    color='[inverse]_unstim_over_DMSO_downreg_scaled', 
    library_key='sample', 
    img=None, 
    title=adata.obs.sample.cat.categories,
    size=8,  # point size
    ncols=4,
    cmap='RdBu_r', vmin=-4, vmax=4, 
    library_id=adata.obs.sample.cat.categories,
    save="spatial/[inverse]_unstim_over_DMSO_downreg_scaled.pdf", dpi=300
)

In [ ]:
# Spatial plots of all perturbation signature scores
rcParams["figure.figsize"] = (6, 6)
sc.set_figure_params(dpi_save=300)

for column in [f'{rstim}_upreg_scaled' for rstim in de_tabs_sig.keys()]:
    print(column)
    sq.pl.spatial_scatter(
        adata, 
        color=column, 
        library_key='sample', 
        img=None, 
        title=adata.obs.sample.cat.categories,
        size=8,  # point size
        ncols=4,
        cmap='RdBu_r', vmin=-4, vmax=4, 
        library_id=adata.obs.sample.cat.categories,
#        save="spatial/spatial_ " + column + ".pdf",
        dpi=300
    )

In [ ]:
#adata.obs.sample

In [ ]:
# # Encode specific cell types as yes/no
# adata.obs['S5'] = pd.NA
# adata.obs.S5[adata.obs.annot == "S5"] = 'Yes'

In [ ]:
# adata.obs['S3_oral_mucosa-like'] = pd.NA
# adata.obs['S3_oral_mucosa-like'][adata.obs.annot == "S3_oral_mucosa-like"] = 'Yes'

In [ ]:
#del adata.uns['S5_colors']

In [ ]:
# rcParams["figure.figsize"] = (6, 6)
# sc.set_figure_params(dpi_save=450)

# sq.pl.spatial_scatter(
#     adata, 
#     color=[
#         'IL1B_upreg_scaled', 'IFNA_upreg_scaled', 'IL17A_upreg_scaled', 'OSM_upreg_scaled', 'IL13_upreg_scaled', 'TNF_upreg_scaled', 'TGFB1_upreg_scaled',
#         'spatial_cluster18', 'S5', 'S3_oral_mucosa-like', 'annot'],
#     crop_coord=(2500, 5500, 4200, 10000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
# #    title=adata.obs.sample.cat.categories,
#     size=6,  # point size
#     ncols=4,
#     cmap='RdBu_r', vmin=-4, vmax=4, 
#     library_id="0033516_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0033516_affected.pdf",
#     dpi=450
# )

In [ ]:
# rcParams["figure.figsize"] = (6, 6)

# sq.pl.spatial_scatter(
#     adata, 
#     color='annot',
#     groups=['S5', 'S3_oral_mucosa-like', 'EC_venous',
#             #'Pericyte_1',
#             #'EC_capillary',
#  #           'B_naive',
#  #           'Enterocyte'
#             'Tnaive/cm_CD4',
#             'B_plasma_IgA1',
#             'B_memory',
#             'Treg_IL10'
#            ],
#     crop_coord=(2500, 5500, 4200, 10000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
#     size=6,  # point size
#     ncols=4,
#     palette="tab10",
#     library_id="0033516_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0033516_affected_selectedCells.pdf",
#     dpi=450
# )

In [ ]:
# adata.obs.sample.value_counts()

In [ ]:
# adata

In [ ]:
# rcParams["figure.figsize"] = (6, 6)
# sc.set_figure_params(dpi_save=300)

# sq.pl.spatial_scatter(
#     adata, 
#     color=[
#         'IL1B_upreg_scaled', 'IFNA_upreg_scaled', 'IL17A_upreg_scaled', 'OSM_upreg_scaled', 'IL13_upreg_scaled', 'TNF_upreg_scaled', 'TGFB1_upreg_scaled',
#         'spatial_cluster18', 'S5', 'S3_oral_mucosa-like', 'annot'],
#     crop_coord=(4500, 3500, 8000, 5000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
# #    title=adata.obs.sample.cat.categories,
#     size=6,  # point size
#     ncols=4,
#     cmap='RdBu_r', vmin=-4, vmax=4, 
#     library_id="0043587_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0043587_affected.pdf",
#     dpi=300
# )

In [ ]:
# rcParams["figure.figsize"] = (6, 6)

# sq.pl.spatial_scatter(
#     adata, 
#     color='annot',
#     groups=['S5',
#             'S3_oral_mucosa-like',
#             'EC_venous',
#   #          'Pericyte_1',
# #             'EC_capillary',
# #             'B_naive',
# #  #           'Enterocyte'
#              'Tnaive/cm_CD4',
#              'B_plasma_IgA1',
#              'B_memory',
#              'Treg_IL10',
#  #           'Tfh', 'S4', 'DC_pDC'
#             'Trm_Th17', 'DC_migratory', 'DC_cDC2'
#            ],
#     crop_coord=(4500, 3500, 8000, 5000),   # x0, y0, x1, y1
#     library_key='sample', 
#     img=None, 
#     size=6,  # point size
#     ncols=4,
#     palette="tab10",
#     library_id="0043587_affected",
#     na_color='lightgrey',
#     save="spatial/cytokines_view1_0043587_affected_selectedCells.pdf",
#     dpi=450
# )